# Deep Q Learning ile Atari Space Invaders oyunu

 <img src="https://github.com/MehmetCanak/Bitirme_tezi/blob/master/spaceinvaders.gif" alt="space" />

Bu not defterinde, OpenAI gym'yi ortam kütüphanesi olarak kullanarak Atari Space Invaders oynamayı 
öğrenen bir ajan uygulayacağız.

# 1. Adım : kütüphanelerin import edilmesi

In [1]:
import tensorflow as tf      # Deep Learning kütüphanesi
import numpy as np           # matriksleri oluşturmak için kütüphane
import gym                   # gym ortamı


from skimage import transform # Çerçeveleri ön işlememize yardımcı olur
from skimage.color import rgb2gray # Çerçevelerimizi grileştirmemize yardımcı olun

import matplotlib.pyplot as plt # Grafikleri görüntüle

from collections import deque

import random

import warnings # This ignore all the warning messages that are normally printed during the training because of skiimage
                #Bu, egzersiz sırasında normal olarak yazdırılan tüm uyarı mesajlarını yok sayma nedeniyle yok sayar.
warnings.filterwarnings('ignore') 

# 2. Adım : Ortamın Oluşturması
Ortamımız Atari Space Invaders oyunudur.

In [2]:
env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)
env.seed(0)

[0, 592379725]

In [3]:
print("Çerçevemizin boyutu: ", env.observation_space)

Çerçevemizin boyutu:  Box(210, 160, 3)


In [4]:
print("hareketin boyutu : ", env.action_space.n)

hareketin boyutu :  6


In [5]:
possible_actions = np.array(np.identity(env.action_space.n,dtype=int).tolist())

In [6]:
print("olası hareketler : ", possible_actions)

olası hareketler :  [[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


# Adım 3: Önişleme işlevlerini tanımlayın ⚙️
preprocess_frame
Önişleme önemli bir adımdır, çünkü eğitim için gereken hesaplama süresini azaltmak için durumlarımızın karmaşıklığını azaltmak istiyoruz.

Adımlarımız:

Çerçevelerimizin her birini gri tonlamalı olmalıdır
Ekranı kırpıyoruz
Piksel değerlerini normalleştiriyoruz
Son olarak, önceden işlenmiş çerçeveyi yeniden boyutlandırıyoruz

In [7]:
""" 
    önişlem çerçevesi:
     Bir çerçeve alıp Gri tonlamalı yapıp yeniden boyutlandıracaz.
        __________________
        |                 |
        |                 |
        |                 |
        |                 |
        |_________________|   
        
        den
        _____________
        |            |
        |            |
        |            |
        |____________|
    e cevirip normalize edeceğiz.
    
    önişlem cercevesi yapıp döndüreceğiz.
    
    """


' \n    önişlem çerçevesi:\n     Bir çerçeve alıp Gri tonlamalı yapıp yeniden boyutlandıracaz.\n        __________________\n        |                 |\n        |                 |\n        |                 |\n        |                 |\n        |_________________|   \n        \n        den\n        _____________\n        |            |\n        |            |\n        |            |\n        |____________|\n    e cevirip normalize edeceğiz.\n    \n    önişlem cercevesi yapıp döndüreceğiz.\n    \n    '

In [8]:
def preprocess_frame(frame):
    # gri tonlamalı cerceve
    gray = rgb2gray(frame)
    
    cropped_frame = gray[8:-12,4:-12]  #kırpılmış cerceve
    
    # Piksel Değerlerini Normallizelestir
    normalized_frame = cropped_frame/255.0
    
    #Yeniden Boyutlandırma

    preprocessed_frame = transform.resize(normalized_frame, [110,84])
    
    return preprocessed_frame # 110x84x1 cerceve

yığın çerçeveler

raporda açıklandığı gibi çerçeveleri yığınlıyoruz.

Çerçeveleri istiflemek gerçekten önemlidir, çünkü Sinir Ağımıza bir hareket duygusu vermemize yardımcı olur.


In [9]:
stack_size = 4 # We stack 4 frames

# Her görüntü için bir dizi olmak üzere sıfır görüntü ile deque başlat
stacked_frames  =  deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
    # Önişlem çerçevesi
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Stacked_frames öğelerini temizleyecez
        stacked_frames = deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        # Çünkü yeni bir bölümdeyiz,aynı çerçeveyi 4x kopyalayacağız
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Deque'ye çerçeve ekle, en eski çerçeveyi otomatik olarak kaldırır
        stacked_frames.append(frame)

        # Yığılmış durumu oluşturma (ilk boyut farklı kareleri belirtir)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

# 4. Adım: Hiperparametreleri ayarlayacaz

Bu bölümde farklı hiperparametrelerimizi kuracağız. Ancak bir Sinir Ağını kendiniz uyguladığınızda,
hiperparamatörleri aynı anda değil, aşamalı olarak uygulayacağız.

İlk olarak, modeli uygularken sinir ağları hiperparametrelerini tanımlayarak başlarız.
Ardından, egzersiz algoritmasını uygularken egzersiz hiperparametrelerini ekleyeceğiz.

In [10]:
### MODEL HYPERPARAMETERLERİ
state_size = [110, 84, 4]      # Girişimiz 4 çerçeveli bir gruptur, 110x84x4 (Genişlik(Width), yükseklik, kanallar) 
action_size = env.action_space.n # 8 possible actions
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 50            # Total episodes for training
max_steps = 50000              # Max possible steps in an episode
batch_size = 64                # Batch size

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.00001           # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.9                    # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

### PREPROCESSING HYPERPARAMETERS
stack_size = 4                 # Number of frames stacked

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

# Adım 5: Derin Q-öğrenme Sinir Ağı modelimizi oluşturuyoruz

Derin Q-öğrenme modelimiz:

Giriş olarak 4 kare yığını alıyoruz

In [11]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(name):
            # placeholders(yer tutucuları) oluşturuyoruz
            # state_size, state_size öğelerinin her öğesini tuple olarak aldığımız anlamına gelir.
            # [None, 84, 84, 4]
            self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.actions_ = tf.placeholder(tf.float32, [None, self.action_size], name="actions_")
            
            #  target_Q is the R(s,a) + ymax Qhat(s', a')
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            """
            ilk Convolutional Neural Network (ConvNet)
            """
            
            # Input is 110x84x4
            self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                         filters = 32,
                                         kernel_size = [8,8],
                                         strides = [4,4],
                                         padding = "VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv1")
            
            self.conv1_out = tf.nn.elu(self.conv1, name="conv1_out")
            
            """
            ikinci Convolutional Neural Network (ConvNet)
            """
            self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                 filters = 64,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv2")

            self.conv2_out = tf.nn.elu(self.conv2, name="conv2_out")            
            
            """
            üçüncü Convolutional Neural Network (ConvNet)
            """
            self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                 filters = 64,
                                 kernel_size = [3,3],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv3")

            self.conv3_out = tf.nn.elu(self.conv3, name="conv3_out")
            
            self.flatten = tf.contrib.layers.flatten(self.conv3_out)
            
            self.fc = tf.layers.dense(inputs = self.flatten,
                                  units = 512,
                                  activation = tf.nn.elu,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                name="fc1")
            
            self.output = tf.layers.dense(inputs = self.fc, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          units = self.action_size, 
                                        activation=None)
            

  
            # Q tahmin edilen Q değerimizdir.
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_))
            
            # Kayıp, tahmin edilen Q_value ve Q_target arasındaki farktır
            # Sum(Qtarget - Q)^2
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
            